# DRYML Tutorial

## Ray tune

One primary use case for DRYML is hyperparameter tuning of your models. `ObjectDef` provides an easy way to define and construct models. The `train` method of trainables allows uniform training for all models, and DRYML provides a set of interface methods to easily run hyperparameters searches on your models.

We'll briefly introduce `ray`, and write a simple hyperparameter tuning example.

RAY is a platform for remote process execution. It creates a server which manages connected resources. Jobs can then be sent to those resources in the form of multiple processes confined to specific resources. This is ideal for hyperparameter tuning, and in fact RAY provides the `ray.tune` library for exactly this.

This tutorial won't serve as a tutorial for ray, for that please consult the ray documentation available here: https://docs.ray.io/en/latest/index.html and here: https://docs.ray.io/en/latest/tune/index.html

Let's start the ray server, and write a simple method for generating models.

In [1]:
import ray

In [2]:
ray.init()

RayContext(dashboard_url='', python_version='3.8.13', ray_version='1.12.1', ray_commit='{{RAY_COMMIT_SHA}}', address_info={'node_ip_address': '192.168.2.31', 'raylet_ip_address': '192.168.2.31', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-09-28_14-52-15_496739_876398/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-09-28_14-52-15_496739_876398/sockets/raylet', 'webui_url': '', 'session_dir': '/tmp/ray/session_2022-09-28_14-52-15_496739_876398', 'metrics_export_port': 58261, 'gcs_address': '192.168.2.31:42538', 'address': '192.168.2.31:42538', 'node_id': '375af3c455875c2c6c0f91a7d97860faef42e97ab1c65a3a3e93200b'})

## Define model generator

def model_def_gen(params):